In [ ]:
from llama_index.core import SimpleDirectoryReader
dir_reader = SimpleDirectoryReader('./data')
documents = dir_reader.load_data()

In [ ]:
from llama_index.core import SimpleDirectoryReader, KnowledgeGraphIndex
from llama_index.core.graph_stores import SimpleGraphStore

from llama_index.llms.openai import OpenAI
from llama_index.core import Settings
from IPython.display import Markdown, display

In [ ]:
llm = OpenAI(temperature=0, model="gpt-3.5-turbo")
Settings.llm = llm
Settings.chunk_size = 512
from llama_index.core import StorageContext

from llama_index.embeddings.openai import OpenAIEmbedding

embed_model = OpenAIEmbedding(model="text-embedding-3-large")



In [ ]:
#use pickle to save the index

import pickle

with open('index.pkl', 'wb') as f:
    pickle.dump(index, f)

#read the pickel

    

In [ ]:
with open('index.pkl', 'rb') as f:
    # Load the data back into memory
    loaded_index = pickle.load(f)

In [ ]:
query_engine = index.as_query_engine(
    include_text=True,
    response_mode="tree_summarize",
    embedding_mode="hybrid",
    similarity_top_k=5,
    verbose=True,
)

response = query_engine.query(
    "According to the Banking Act, when can distributions be made following the removal of a licensed commercial bank's name from the list of licensed commercial banks?"
)
display(Markdown(f"<b>{response}</b>"))

In [ ]:

query_engine.get_prompts()

In [ ]:
from pyvis.network import Network

g = index.get_networkx_graph()
net = Network(notebook=True, cdn_resources="in_line", directed=True)
net.from_nx(g)
net.show("example.html")

In [ ]:
from flask import Flask, jsonify, request, json
from flask_cors import CORS

app = Flask(__name__)
CORS(app)

@app.route('/status', methods=['GET'])
def get_status():
    #return status ok
    return jsonify({'status': 'ok'}), 200


#add a POST route at /chat. Get the request body and echo the message back
@app.route('/chat', methods=['POST'])
def chat():
    data = json.loads(request.data)
    
    #get the last message from the user
    message = data['messages'][-1]['content']
    print(message)
    response = str(query_engine.query(str(message)))
    print(response)
    #message = data['messages']
    return jsonify({'status': 'ok', 'response': response}), 200

if __name__ == '__main__':
    app.run(port=3000)
